In [ ]:
from PIL import ImageDraw, ImageFont
import random
import matplotlib.pyplot as plt
import io
import numpy as np    
from PIL import Image
import time
import cv2

ww = 32
hh = 32

def generateRandomImage():
    # create Image object with the input image
    font = ImageFont.truetype('/content/drive/My Drive/KPMG/boldTimes.TTF', size=26)
    # draw the message on the background
    labels = 'abcdefghijklmnopqrstuvwxyz0123456789'
    message = ''
    rois = []
    c = 0
    while not c == 4:
      im = Image.new(mode = "RGB", size = (150, 50), color = (255, 255, 255))
      draw = ImageDraw.Draw(im)
      (x, y) = (30, 8)
      r1 = random.randint(0, 35)
      r2 = random.randint(0, 35)
      r3 = random.randint(0, 35)
      r4 = random.randint(0, 35)
      message = "%s%s%s%s" % (labels[r1], labels[r2], labels[r3], labels[r4])

      positions = np.zeros((4, 36))
      positions[0][r1] = 1
      positions[1][r2] = 1
      positions[2][r3] = 1
      positions[3][r4] = 1

      #positions.append(labels.index(i))
      for i in message:
          w, h = draw.textsize(i, font=font)
          dx = random.randint(-4, 4)
          dy = random.randint(-10, 10)
          space = random.randint(0, 5)
          
          #out.extend([x + dx, y + dy, x + dx + w,  y + dy + h])
          draw.text((x + dx, y + dy), i, fill=(0,0,0,0), font=font)
          x += w + space
          

      img = np.array(im)
      #im2arr = np.array(im)
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      ret, thresh = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY)
      ctrs, hier = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
      sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
      c = 0
      rois = []
      for i, ctr in enumerate(sorted_ctrs):
          x, y, w, h = cv2.boundingRect(ctr)
          area = w*h
          if 120 < area < 500:
            rois.append(img[y-3:y + h +3, x-1:x + w + 2])
            c += 1

    for i in range(len(rois)):
        rois[i] = cv2.resize(rois[i], dsize=(ww, hh), interpolation=cv2.INTER_CUBIC)

    return rois, np.array(positions)

def myGenerator():
    while True:
        batch_features = np.zeros((4, hh, ww, 3), dtype="float64")
        batch_labels = np.zeros((4, 36), dtype="float64")
        img, b = generateRandomImage()

        for i in range(len(img)):
            batch_features[i] = img[i]
            batch_labels[i] = b[i]


        yield batch_features/255, batch_labels

im,  positions = next(myGenerator())

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K


# Import Tensorflow with multiprocessing
import tensorflow as tf
import multiprocessing as mp


batch_size = 4
# 32 examples in a mini-batch, smaller batch size means more updates in one epoch
num_classes = 36 #
epochs = 100 # repeat 100 times

def base_model():

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr = 0.001, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model
cnn_n = base_model()
cnn_n.summary()




Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_30 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_31 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 15, 15, 64)      

In [ ]:
cnn_n.fit_generator(
    myGenerator(),
    steps_per_epoch=2000,
    epochs=50,
    validation_data=myGenerator(),
    validation_steps=800)

Epoch 1/50
2000/2000 [==============================] - 20s 10ms/step - loss: 0.8578 - accuracy: 0.7741 - val_loss: 8.0466e-07 - val_accuracy: 0.9997
Epoch 2/50
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0246 - accuracy: 0.9958 - val_loss: 1.2845e-05 - val_accuracy: 0.9991
Epoch 3/50
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0126 - accuracy: 0.9981 - val_loss: 1.1921e-07 - val_accuracy: 0.9984
Epoch 4/50
2000/2000 [==============================] - 20s 10ms/step - loss: 0.0168 - accuracy: 0.9976 - val_loss: 2.8401e-05 - val_accuracy: 0.9994
Epoch 5/50
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0077 - accuracy: 0.9984 - val_loss: 1.4901e-07 - val_accuracy: 0.9984
Epoch 6/50
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0095 - accuracy: 0.9981 - val_loss: 3.7253e-06 - val_accuracy: 0.9991
Epoch 7/50
2000/2000 [==============================] - 19s 10ms/step - loss: 0.0148 - accuracy: 0.9

In [ ]:
cnn_n.save('/content/drive/My Drive/KPMG/model1')



In [ ]:
cnn_n.save_weights("/content/drive/My Drive/KPMG/ckpt")